### Linear Arrays

#### General functions

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rc
from matplotlib.colors import LogNorm

light_speed = 2.99792458e8
mu_0 = 4*np.pi*1e-7
epsilon_0 = 1/(mu_0*light_speed**2)

Define function to plot contours in 2D plane.

In [ ]:
def surface_plot_three_panel(X, Y, Z, XY, XZ, YZ):
    # set up latex labels on plot (optional)
    try:
        rc('text', usetex=True) # can try usetex=False
        rc('font', size=12)
        rc('legend', fontsize=12)
        rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
    except:
        pass    
    
    # create figure
    fig, ax = plt.subplots(1, 3, figsize=(14, 4))
    
    # set axis labels and limits
    ax[0].set_xlabel(r'$x (m)$')
    ax[0].set_ylabel(r'$y (m)$')
    ax[0].set_aspect(1)
    ax[1].set_xlabel(r'$y (m)$')
    ax[1].set_ylabel(r'$z (m)$')
    ax[1].set_aspect(1)
    ax[2].set_xlabel(r'$z (m)$')
    ax[2].set_ylabel(r'$x (m)$')
    ax[2].set_aspect(1)
    
    # plot surface map
    lims = np.concatenate((XY.flatten(), XZ.flatten(), YZ.flatten()))
    ax[0].pcolor(X[:,:,0], Y[:,:,0], XY, norm=LogNorm(vmin=np.nanquantile(lims, 0.16), vmax=np.nanquantile(lims, 0.975)))
    ax[1].pcolor(Y[0,:,:], Z[0,:,:], YZ, norm=LogNorm(vmin=np.nanquantile(lims, 0.16), vmax=np.nanquantile(lims, 0.975)))
    ax[2].pcolor(Z[:,0,:], X[:,0,:], np.transpose(XZ), norm=LogNorm(vmin=np.nanquantile(lims, 0.16), vmax=np.nanquantile(lims, 0.975)))
    
    plt.show()  

    return fig

#### Workshop functions

Define function to calculate the field components of a half-wave dipole.

In [ ]:
def half_wave_dipole(radius, theta, phase=0., frequency=1e9, time=0., i0=1., kappa_e=1., kappa_m=1.):

    # calculate the phase velocity
    phase_velocity = light_speed/np.sqrt(kappa_e*kappa_m)

    # calculate electric and magnetic field components
    E_theta = ...
    B_phi = ...

    return E_theta, B_phi

Define function to calculate the field pattern from the dipole in cartesian coordinates - there is logic here.

In [ ]:
def half_wave_dipole_cartesian(x, y, z, phase=0., frequency=1e9, time=0., i0=1., kappa_e=1., kappa_m=1.):

    # convert input cartesian coordinates into spherical coordinates
    radius = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z/radius)
    phi = np.arctan2(y, x)

    # call spherical function to find electric and magnetic fields
    E_theta, B_phi = half_wave_dipole(radius, theta, phase=phase, frequency=frequency, time=time, i0=i0, kappa_e=kappa_e, kappa_m=kappa_m)

    # convert electric and magnetic field vectors back to cartesian coordinates
    E_x, E_y, E_z = ..., ..., ...
    B_x, B_y, B_z = -B_phi*np.sin(phi), B_phi*np.cos(phi), B_phi*0

    return np.asarray([E_x, E_y, E_z]), np.asarray([B_x, B_y, B_z])

Define function to calculate field contribution from an array of dipoles with given $(x,y,z)$ locations and phase offsets.

In [ ]:
def dipole_array(x, y, z, dipole_x, dipole_y, dipole_z, dipole_phase, frequency=1e9, time=0., i0=1., kappa_e=1., kappa_m=1.):

    # create vectors to store field components
    E_vector = np.zeros(3)
    B_vector = np.zeros(3)
    S_vector = np.zeros(3)
    
    # sum field contributions from each dipole in the array
    if not isinstance(dipole_x, (list, np.ndarray)):
        dipole_x, dipole_y, dipole_z, dipole_phase = [dipole_x], [dipole_y], [dipole_z], [dipole_phase]
    for i in range(0, len(dipole_x)):
        E, B = half_wave_dipole_cartesian(x - dipole_x[i], y - dipole_y[i], z - dipole_z[i], phase=dipole_phase[i], \
                        frequency=frequency, time=time, i0=i0, kappa_e=kappa_e, kappa_m=kappa_m)
        E_vector = E_vector + E
        B_vector = B_vector + B

    # calculate power from field components
    S_vector[0] = ...
    S_vector[1] = ...
    S_vector[2] = ...

    return E_vector, B_vector, S_vector

In [ ]:
def plot_radiation_pattern(dipole_x, dipole_y, dipole_z, dipole_phase, frequency=1e9, time=0., i0=1., kappa_e=1., kappa_m=1.):

    # calculate the wavelength
    wavelength = light_speed/frequency

    # create meshgrids
    points = np.linspace(-100*wavelength, 100*wavelength + 1e-9, 200)
    X, Y, Z = np.meshgrid(points, points, points, indexing='ij')
    
    # plot in xy-plane
    power_xy = np.zeros_like(X[:,:,0])
    for i in range(0, len(X[:,0,0])):
        for j in range(0, len(Y[0,:,0])):
            E_vector, B_vector, S_vector = dipole_array(X[i,j,0], Y[i,j,0], 0, dipole_x, dipole_y, dipole_z, dipole_phase, frequency=frequency)
            power_xy[i, j] = np.abs(S_vector[0]) + np.abs(S_vector[1]) + np.abs(S_vector[2])   
    
    # plot in xz-plane
    power_xz = np.zeros_like(X[:,0,:])
    for i in range(0, len(X[:,0,0])):
        for j in range(0, len(Z[0,0,:])):
            E_vector, B_vector, S_vector = dipole_array(X[i,0,j], 0, Z[i,0,j], dipole_x, dipole_y, dipole_z, dipole_phase, frequency=frequency)
            power_xz[i, j] = np.abs(S_vector[0]) + np.abs(S_vector[1]) + np.abs(S_vector[2])  

    # plot in yz-plane
    power_yz = np.zeros_like(X[0,:,:])
    for i in range(0, len(Y[0,:,0])):
        for j in range(0, len(Z[0,0,:])):
            E_vector, B_vector, S_vector = dipole_array(0, Y[0,i,j], Z[0,i,j], dipole_x, dipole_y, dipole_z, dipole_phase, frequency=frequency)
            power_yz[i, j] = np.abs(S_vector[0]) + np.abs(S_vector[1]) + np.abs(S_vector[2]) 

    # create three panel plot of radiated power
    fig = surface_plot_three_panel(X, Y, Z, power_xy, power_xz, power_yz)

    return fig

### Parallel half-wave dipoles space by $\lambda/2$ (along $x$-axis)

Set location of dipoles, and their relative phases.

In [ ]:
frequency = 1e9
wavelength = light_speed/frequency

In [ ]:
dipole_x = [-wavelength/4, wavelength/4]
dipole_y = [0, 0]
dipole_z = [0, 0]
dipole_phase = [0, 0]

In [ ]:
plot_radiation_pattern(dipole_x, dipole_y, dipole_z, dipole_phase)

### Out-of-phase parallel half-wave dipoles space by $\lambda/2$ (along $x$-axis)

In [ ]:
dipole_x = [-wavelength/4, wavelength/4]
dipole_y = [0, 0]
dipole_z = [0, 0]
dipole_phase = [-np.pi/2, np.pi/2]

In [ ]:
plot_radiation_pattern(dipole_x, dipole_y, dipole_z, dipole_phase)

### Array of eight parallel half-wave dipoles space by $\lambda/2$ (along $x$-axis)

In [ ]:
dipole_x = [-7*wavelength/4, -5*wavelength/4, -3*wavelength/4, -wavelength/4, wavelength/4, 3*wavelength/4, 5*wavelength/4, 7*wavelength/4]
dipole_y = [0, 0, 0, 0, 0, 0, 0, 0]
dipole_z = [0, 0, 0, 0, 0, 0, 0, 0]
delta = 2*np.pi/wavelength*(wavelength/2)*np.sin(0*np.pi/2)
dipole_phase = [0, delta, 2*delta, 3*delta, 4*delta, 5*delta, 6*delta, 7*delta]

In [ ]:
plot_radiation_pattern(dipole_x, dipole_y, dipole_z, dipole_phase)